# Bayesian Sparse Feature Selection Demo

This notebook demonstrates the Bayesian Gaussian Mixture Feature Selection algorithm on artificial data.

**Features:**
- Modular prior (spike-and-slab, easily replaceable)
- Flexible mixture model
- Variational inference with PyTorch
- Interactive diagnostics with Plotly/Seaborn

> **Note:** Ensure that the `feature_selection` package and dependencies are installed.

In [ ]:
# %pip install -q -e ..

In [ ]:
# Add parent directory to Python path to find our package
import sys
import os
from pathlib import Path

parent_dir = Path(os.path.dirname(os.getcwd()))
if parent_dir not in sys.path:
    sys.path.insert(0, str(parent_dir))

In [ ]:
from typing import Dict, List
from IPython.display import display, Markdown
import numpy as np
import pandas as pd
import torch
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

import feature_selection.config as C
from feature_selection.utils import (
    solve_with_logistic_regression,
    solve_with_linear_regression,
    recover_solutions,
    display_features_overview,
    show_regression_results_for_solutions,
    show_algorithm_progress,
)
from feature_selection.inference import BayesianFeatureSelector
from feature_selection.visualizations import (
    plot_elbo,
    plot_mu,
    plot_alpha,
    show_correlation_matrix,
    show_correlations_with_response,
    show_label_histogram,
    show_features_in_components,
    compare_parameters,
)
from feature_selection.generate_artificial_dataset import (
    generate_artificial_dataset,
)

pio.renderers.default = "notebook_connected"  # Ensures plotly plots show in notebooks

# Import parameters and settings

# Generate Artificial Dataset

In [ ]:
# Generate dataset
df, y, generating_solutions, parameters = generate_artificial_dataset(
    n_samples=C.NSAMPLES,
    n_features=C.NFEATURES,
    n_solutions=C.NSOLUTIONS,
    sparsity=C.SPARSITY,
    noise_data_std=C.NOISE_STD,
    binarize=C.BINARIZE,
    binary_response_ratio=C.BINARY_RESPONSE_RATIO,
    random_seed=C.RANDOM_SEED,
    save_to_csv=False,
    print_data_overview=True,
)

support_indices = parameters["support_indices"].sum()
true_support_features = [f"feature_{i}" for i in set(support_indices)]

# Classical approach
Solve the classification problem using logistic regression with l1 penalty

In [ ]:
# Solve the full problem (all features) with l2 penalty
#
# if BINARIZE:
#     solve_with_logistic_regression(X=df, y=y, penalty="l2")
# else:
#     solve_with_linear_regression(X=df, y=y, penalty="l2")

In [ ]:
# Solve the full problem (all features) with l1 penalty
if C.BINARIZE:
    solve_with_logistic_regression(X=df, y=y, penalty="l1")
else:
    solve_with_linear_regression(X=df, y=y, penalty="l1")

In [ ]:
# Show regression results only for the generating solutions
show_regression_results_for_solutions(
    solutions=generating_solutions,
    df=df,
    y=y,
    penalty="l1",
)

In [ ]:
# Solve for arbitrary combinations of features
combinations = [
    # ["3", "17", "30", "39"],
]

for combination in combinations:
    display(Markdown(f"## Regression model for component {combination}"))
    features = [f"feature_{i}" for i in combination]
    display(Markdown(f"- Support features: {features}"))
    if C.BINARIZE:
        solve_with_logistic_regression(X=df[features], y=y, penalty="l1")
    else:
        solve_with_linear_regression(X=df[features], y=y, penalty="l1")
    display(Markdown("------------------"))

# Bayesian Feature Selector

In [ ]:
selector = BayesianFeatureSelector(
    n_features=C.NFEATURES,
    n_components=C.N_COMPONENTS,
    X=df.values,
    y=y,
    prior=C.PRIOR_TYPE,
    sss_sparsity=C.PRIOR_SPARSITY,
    var_slab=C.VAR_SLAB,
    var_spike=C.VAR_SPIKE,
    weight_slab=C.WEIGHT_SLAB,
    weight_spike=C.WEIGHT_SPIKE,
    student_df=C.STUDENT_DF,
    student_scale=C.STUDENT_SCALE,
    lr=C.LEARNING_RATE,
    batch_size=C.BATCH_SIZE,
    n_iter=C.N_ITER,
)

history = selector.optimize(
    regularize=C.IS_REGULARIZED,
    lambda_jaccard=C.LAMBDA_JACCARD,
    verbose=True,
)

show_algorithm_progress(history)

## Solution quality assessment

In [ ]:
solutions, final_parameters, full_nonzero_solutions = recover_solutions(
    search_history=history,
    desired_sparsity=C.DESIRED_SPARSITY,
    min_mu_threshold=C.MIN_MU_THRESHOLD,
    verbose=False,
)

## Overview of discovered features

In [ ]:
features_found = set().union(
    *solutions.values()
)  # unique features occuring in any component


display_features_overview(
    features_found=features_found,
    true_support_features=true_support_features,
    n_total_features=len(df.columns),
)

## Comparison of the solutions

In [ ]:
display(pd.DataFrame.from_dict(solutions, orient="index").T)

In [ ]:
features_to_show = list(set(true_support_features).union(set(features_found)))
show_features_in_components(solutions, features_to_show=features_to_show)

show_correlation_matrix(df[sorted(features_to_show)], width=600, height=600)

## Mixture means vs. true means

In [ ]:
# Show final mixture means and weights
compare_parameters(parameters, final_parameters["final mu"])

display(Markdown("### Final mixture weights (alpha):"))
for i, alpha in enumerate(final_parameters["final alpha"]):
    display(Markdown(f"- **Component {i}:** {alpha:.3f}"))